# 🎲 Monte Carlo Integration
### Comparing numerical approximation vs exact values — with error convergence plots

We'll integrate two functions:
1. **Simple:** $f(x) = x^2$ on $[0, 3]$ — exact answer: $9$
2. **Nasty:** $f(x) = \cos(x) \cdot e^{-x^2/2} \cdot \sqrt{|W(|x|+1)|}$ — no closed form, numerical ground truth via `scipy.integrate.quad`

where $W$ is the Lambert W function (the inverse of $x e^x$).

Monte Carlo integration exploits the identity:
$$\int_a^b f(x)\, dx = (b-a) \cdot \mathbb{E}[f(X)], \quad X \sim \text{Uniform}(a, b)$$

As $N \to \infty$, the error goes as $\sim 1/\sqrt{N}$.

In [5]:
%matplotlib inline

import matplotlib

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.integrate import quad
from scipy.special import lambertw

plt.style.use('dark_background')
ACCENT1 = '#00d4ff'   # cyan
ACCENT2 = '#ff6b6b'   # red-pink
ACCENT3 = '#ffd93d'   # yellow
ACCENT4 = '#6bcb77'   # green
print('Imports done. Let\'s go.')

Imports done. Let's go.


## 1. Define the Functions

In [6]:
# ── Simple function ──────────────────────────────────────────────
def f_simple(x):
    """f(x) = x^2"""
    return x**2

A_SIMPLE, B_SIMPLE = 0.0, 3.0
EXACT_SIMPLE = 9.0  # ∫₀³ x² dx = [x³/3]₀³ = 9

# ── Nasty function (Lambert W edition) ──────────────────────────
def f_nasty(x):
    """
    f(x) = cos(x) * exp(-x²/2) * sqrt(W(|x| + 1))

    W is the Lambert W function: inverse of z = w·e^w
    This has no elementary antiderivative.
    """
    W_val = np.real(lambertw(np.abs(x) + 1))   # principal branch, always real here
    return np.cos(x) * np.exp(-x**2 / 2) * np.sqrt(W_val)

A_NASTY, B_NASTY = -3.0, 3.0

# Ground truth via adaptive quadrature (scipy)
EXACT_NASTY, quad_err = quad(f_nasty, A_NASTY, B_NASTY, limit=200)
print(f'Exact  ∫ x² dx  over [{A_SIMPLE}, {B_SIMPLE}]  =  {EXACT_SIMPLE}')
print(f'Quad   ∫ nasty  over [{A_NASTY}, {B_NASTY}]  ≈  {EXACT_NASTY:.10f}  (quad err ≈ {quad_err:.2e})')

Exact  ∫ x² dx  over [0.0, 3.0]  =  9.0
Quad   ∫ nasty  over [-3.0, 3.0]  ≈  1.2690963429  (quad err ≈ 1.13e-12)


## 2. Visualise the Functions

In [7]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Functions to Integrate', fontsize=15, color='white', y=1.02)

# Simple
xs = np.linspace(A_SIMPLE, B_SIMPLE, 500)
axes[0].plot(xs, f_simple(xs), color=ACCENT1, lw=2.5)
axes[0].fill_between(xs, f_simple(xs), alpha=0.25, color=ACCENT1)
axes[0].set_title(r'$f(x) = x^2$  on  $[0, 3]$', color='white')
axes[0].set_xlabel('x'); axes[0].set_ylabel('f(x)')
axes[0].text(0.5, 0.85, f'Exact integral = {EXACT_SIMPLE}',
             transform=axes[0].transAxes, color=ACCENT3, ha='center', fontsize=11)

# Nasty
xn = np.linspace(A_NASTY, B_NASTY, 1000)
axes[1].plot(xn, f_nasty(xn), color=ACCENT2, lw=2.5)
axes[1].fill_between(xn, f_nasty(xn), alpha=0.25, color=ACCENT2)
axes[1].set_title(r'$f(x)=\cos(x)\,e^{-x^2/2}\,\sqrt{W(|x|+1)}$  on  $[-3, 3]$', color='white')
axes[1].set_xlabel('x'); axes[1].set_ylabel('f(x)')
axes[1].text(0.5, 0.10, f'Numerical integral ≈ {EXACT_NASTY:.6f}',
             transform=axes[1].transAxes, color=ACCENT3, ha='center', fontsize=11)

for ax in axes:
    ax.axhline(0, color='gray', lw=0.5)
    ax.grid(alpha=0.2)

plt.tight_layout()
plt.show()

<Figure size 1400x500 with 2 Axes>

<Figure size 1400x500 with 2 Axes>

## 3. Monte Carlo Integrator

In [8]:
def monte_carlo_integrate(func, a, b, n_samples, seed=42):
    """
    Estimate ∫ₐᵇ f(x) dx using n_samples uniform random samples.
    Returns the estimate.
    """
    rng = np.random.default_rng(seed)
    x = rng.uniform(a, b, n_samples)
    return (b - a) * np.mean(func(x))


def mc_error_curve(func, a, b, exact, n_values, n_trials=20, seed=0):
    """
    For each N in n_values, run n_trials independent MC estimates.
    Returns:
        mean_errors   — mean |estimate - exact| across trials
        std_errors    — std of absolute errors (for shaded band)
        all_estimates — shape (n_trials, len(n_values)) raw estimates
    """
    rng = np.random.default_rng(seed)
    all_estimates = np.zeros((n_trials, len(n_values)))

    for t in range(n_trials):
        for j, n in enumerate(n_values):
            x = rng.uniform(a, b, n)
            all_estimates[t, j] = (b - a) * np.mean(func(x))

    abs_errors = np.abs(all_estimates - exact)
    return abs_errors.mean(axis=0), abs_errors.std(axis=0), all_estimates


# Sample counts — log-spaced from 10 to 1,000,000
N_VALUES = np.unique(np.logspace(1, 6, 60).astype(int))
N_TRIALS = 30

print('Computing error curves ...')
err_simple_mean, err_simple_std, est_simple = mc_error_curve(
    f_simple, A_SIMPLE, B_SIMPLE, EXACT_SIMPLE, N_VALUES, N_TRIALS)

err_nasty_mean, err_nasty_std, est_nasty = mc_error_curve(
    f_nasty, A_NASTY, B_NASTY, EXACT_NASTY, N_VALUES, N_TRIALS)

print('Done!')

Computing error curves ...
Done!


## 4. Error vs N  —  The Main Event 📉

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Monte Carlo Integration: Absolute Error vs Sample Count N',
             fontsize=14, color='white', y=1.01)

def plot_error_curve(ax, n_vals, mean_err, std_err, color, title, exact_label):
    # Theoretical 1/√N reference line (scaled to match)
    scale = mean_err[5] * np.sqrt(n_vals[5])   # calibrate at a mid point
    ref_line = scale / np.sqrt(n_vals)

    # Shaded trial spread
    ax.fill_between(n_vals,
                    np.maximum(mean_err - std_err, 1e-10),
                    mean_err + std_err,
                    alpha=0.2, color=color, label='±1 std (across trials)')

    ax.loglog(n_vals, mean_err, color=color, lw=2.5, label='Mean |error|')
    ax.loglog(n_vals, ref_line, '--', color=ACCENT3, lw=1.5, alpha=0.8,
              label=r'$\sim 1/\sqrt{N}$ reference')

    ax.set_xlabel('N  (number of samples)', fontsize=12)
    ax.set_ylabel('Absolute Error', fontsize=12)
    ax.set_title(title, color='white', fontsize=12)
    ax.legend(fontsize=9)
    ax.grid(True, which='both', alpha=0.15)
    ax.text(0.97, 0.92, f'True value: {exact_label}',
            transform=ax.transAxes, ha='right', color=ACCENT3, fontsize=10)

plot_error_curve(axes[0], N_VALUES, err_simple_mean, err_simple_std,
                 ACCENT1, r'$f(x)=x^2$ on $[0,3]$', f'{EXACT_SIMPLE}')

plot_error_curve(axes[1], N_VALUES, err_nasty_mean, err_nasty_std,
                 ACCENT2, r'$f(x)=\cos(x)\,e^{-x^2/2}\,\sqrt{W(|x|+1)}$ on $[-3,3]$',
                 f'{EXACT_NASTY:.6f}')

plt.tight_layout()
plt.show()

print('Notice both curves hug the 1/√N slope — that\'s the Central Limit Theorem at work.')

## 5. Estimate Distribution at Fixed N — Histogram

In [ ]:
# Pick a few N values and show distribution of MC estimates across trials
HIGHLIGHT_N = [100, 1_000, 10_000, 100_000]
N_HIST_TRIALS = 2000

fig, axes = plt.subplots(2, 4, figsize=(18, 8))
fig.suptitle('Distribution of MC Estimates Across 2000 Trials', fontsize=13, color='white')

rng = np.random.default_rng(99)

for col, n in enumerate(HIGHLIGHT_N):
    for row, (func, a, b, exact, color, fname) in enumerate([
        (f_simple, A_SIMPLE, B_SIMPLE, EXACT_SIMPLE, ACCENT1, r'$x^2$'),
        (f_nasty,  A_NASTY,  B_NASTY,  EXACT_NASTY,  ACCENT2, 'nasty')
    ]):
        samples = np.array([
            (b - a) * np.mean(func(rng.uniform(a, b, n)))
            for _ in range(N_HIST_TRIALS)
        ])
        ax = axes[row, col]
        ax.hist(samples, bins=60, color=color, alpha=0.75, edgecolor='none')
        ax.axvline(exact, color=ACCENT3, lw=2, linestyle='--', label=f'True = {exact:.4f}')
        ax.axvline(samples.mean(), color='white', lw=1.5, linestyle=':', label=f'MC mean = {samples.mean():.4f}')
        ax.set_title(f'{fname},  N={n:,}', color='white', fontsize=9)
        ax.set_xlabel('Estimate'); ax.set_ylabel('Count' if col == 0 else '')
        ax.legend(fontsize=7)
        ax.grid(alpha=0.15)

plt.tight_layout()
plt.show()

## 6. Live Convergence Animation  🎬
Watch the running estimate zero in on the true value as N grows.

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def running_mc_estimate(func, a, b, n_max, seed=7):
    """Return running MC estimate at every step."""
    rng = np.random.default_rng(seed)
    x = rng.uniform(a, b, n_max)
    fx = func(x)
    running = (b - a) * np.cumsum(fx) / np.arange(1, n_max + 1)
    return running

N_ANIM = 5000
run_simple = running_mc_estimate(f_simple, A_SIMPLE, B_SIMPLE, N_ANIM)
run_nasty  = running_mc_estimate(f_nasty,  A_NASTY,  B_NASTY,  N_ANIM)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Live Convergence of MC Estimate', color='white', fontsize=13)

lines, hlines = [], []
for ax, run, exact, color, title in zip(
    axes,
    [run_simple, run_nasty],
    [EXACT_SIMPLE, EXACT_NASTY],
    [ACCENT1, ACCENT2],
    [r'$x^2$ on $[0,3]$', r'Nasty on $[-3,3]$']
):
    line, = ax.plot([], [], color=color, lw=1.5)
    ax.axhline(exact, color=ACCENT3, lw=1.5, ls='--', label=f'True = {exact:.5f}')
    ax.set_xlim(1, N_ANIM); ax.set_ylim(exact * 0.7, exact * 1.3)
    ax.set_xlabel('N'); ax.set_ylabel('Running Estimate')
    ax.set_title(title, color='white'); ax.legend(fontsize=9); ax.grid(alpha=0.2)
    lines.append(line)

def update(frame):
    idx = frame + 1
    for line, run in zip(lines, [run_simple, run_nasty]):
        line.set_data(np.arange(1, idx + 1), run[:idx])
    return lines

FRAMES = np.unique(np.concatenate([
    np.arange(0, 100),
    np.linspace(100, N_ANIM - 1, 150).astype(int)
]))

anim = FuncAnimation(fig, update, frames=FRAMES, interval=30, blit=True)
plt.close()
HTML(anim.to_jshtml())

## 7. Summary Table

In [ ]:
print(f'{'N':>10}  |  {'x²  estimate':>14}  {'error':>12}  ||  {'nasty estimate':>16}  {'error':>12}')
print('-' * 80)
rng = np.random.default_rng(0)
for n in [10, 100, 1_000, 10_000, 100_000, 1_000_000]:
    xs = rng.uniform(A_SIMPLE, B_SIMPLE, n)
    xn = rng.uniform(A_NASTY,  B_NASTY,  n)
    est_s = (B_SIMPLE - A_SIMPLE) * np.mean(f_simple(xs))
    est_n = (B_NASTY  - A_NASTY)  * np.mean(f_nasty(xn))
    print(f'{n:>10,}  |  {est_s:>14.6f}  {abs(est_s - EXACT_SIMPLE):>12.2e}  ||  {est_n:>16.8f}  {abs(est_n - EXACT_NASTY):>12.2e}')

print(f'{'Exact':>10}  |  {EXACT_SIMPLE:>14.6f}  {'—':>12}  ||  {EXACT_NASTY:>16.8f}  {'—':>12}')

---
## Key Takeaways

- **Monte Carlo integration** converges at $O(1/\sqrt{N})$ regardless of dimension — making it uniquely powerful in high dimensions.
- The **Lambert W function** shows up in problems like `x·eˣ = c` and makes analytic integration essentially impossible here.
- Both simple and complex functions converge at the same **asymptotic rate** — the constant differs based on the variance of $f$ under the sampling distribution.
- The **histograms** illustrate the CLT: as $N$ grows, the distribution of estimates tightens symmetrically around the true value.